Company XYZ has started a new referral program on `Oct, 31.` Each user who refers a new user will get 10$ in credit when the new user buys something. The program has been running for almost a month and the Growth Product Manager wants to know if it's been successful. She is very excited cause, since the referral program started, the company saw a spike in number of users and wants you to be able to give her some data she can show to her boss.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss

plt.style.use('ggplot')

# Load Dataset

In [2]:
data = pd.read_csv('../DS_Challenges/User_Referral_Program/referral.csv', parse_dates = ['date'])
data.head()

,user_id,date,country,money_spent,is_referral,device_id
0,2,2015-10-03,FR,65,0,EVDCJTZMVMJDG
1,3,2015-10-03,CA,54,0,WUBZFTVKXGQQX
2,6,2015-10-03,FR,35,0,CBAPCJRTFNUJG
3,7,2015-10-03,UK,73,0,PRGXJZAJKMXRH
4,7,2015-10-03,MX,35,0,PRGXJZAJKMXRH


In [3]:
# device_id here is not so important so we delete it
del data['device_id']
data['weekday'] = data.date.dt.weekday

In [4]:
data.date.describe()

<ipython-input-4-2cb626a5e644>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.date.describe()


count                   97341
unique                     56
top       2015-11-14 00:00:00
freq                     3303
first     2015-10-03 00:00:00
last      2015-11-27 00:00:00
Name: date, dtype: object

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      97341 non-null  int64         
 1   date         97341 non-null  datetime64[ns]
 2   country      97341 non-null  object        
 3   money_spent  97341 non-null  int64         
 4   is_referral  97341 non-null  int64         
 5   weekday      97341 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 4.5+ MB


In [6]:
data.describe()

,user_id,money_spent,is_referral,weekday
count,97341.000000,97341.000000,97341.000000,97341.000000
mean,6354.801029,44.692360,0.287823,3.804307
std,5682.991186,22.791839,0.452751,2.043796
min,1.000000,10.000000,0.000000,0.000000
25%,2020.000000,27.000000,0.000000,2.000000
50%,4053.000000,42.000000,0.000000,5.000000
75%,10286.000000,59.000000,1.000000,6.000000
max,20000.000000,220.000000,1.000000,6.000000


In [7]:
data.sample(3)

,user_id,date,country,money_spent,is_referral,weekday
20501,1683,2015-10-14,UK,57,0,2
50334,18418,2015-10-31,MX,88,1,5
10693,176,2015-10-09,US,48,0,4


# Question 1:

### Can you estimate the impact the program had on the site

To further explain the question:
- Question 1 will be solved if we compare the user performance before and after the first day of this referral program (Oct 31th). 

In [8]:
# step 1, find the number of days before and after the first of testing
before = pd.to_datetime('2015-10-31') - data.date.min()
after = data.date.max() - pd.to_datetime('2015-10-30')
print(f'first day: {data.date.min()}, before: {before} \nlast day: {data.date.max()}, after: {after} ')

first day: 2015-10-03 00:00:00, before: 28 days 00:00:00 
last day: 2015-11-27 00:00:00, after: 28 days 00:00:00 


In [23]:
# step 2 Hypothesis testing (compare the p-value and other descriptive statistics)
def count_spent(df):
    d = {}
    d['n_purchase'] = df.shape[0]
    d['n_customer'] = df['user_id'].unique().shape[0]
    d['total_spent'] = df['money_spent'].sum()
    return pd.Series(d)

def daily_statistics(df):
    """
    given a dataframe
    1.  group by day, and return '#purchase','total spent money','#customers' on each day
    2.  split daily data into two groups, before the program and after the program
    3.  for each 'sale index' ('#purchase','total spent money','#customers'), 
        calculate the mean before/after the program, their difference, and pvalue 
    """
    referral_start_day = pd.to_datetime('2015-10-31')
    
    grp_days = df.groupby('date').apply(count_spent)
    grp_days_before = grp_days.loc[grp_days.index < referral_start_day]
    grp_days_after = grp_days.loc[grp_days.index >= referral_start_day]
    
    index_name = ['n_purchase','n_customer','total_spent']
    d = pd.DataFrame(index = index_name, columns = ['avg_before','avg_after','avg_diff','p_value'])
    for col in index_name:
        d['avg_before'][col] =  grp_days_before[col].mean()
        d['avg_after'][col] = grp_days_after[col].mean()
        d['avg_diff'][col] = d['avg_after'][col] - d['avg_before'][col]
        d['p_value'][col] = ss.ttest_ind(grp_days_after[col], grp_days_before[col], equal_var = False).pvalue/2
    return d 

In [24]:
# step 3: show the results
daily_statistics(data)

,avg_before,avg_after,avg_diff,p_value
n_purchase,1690.75,1785.71,94.9643,0.348257
n_customer,1384.46,1686.96,302.5,0.0595455
total_spent,71657,83714.4,12057.4,0.135194


although after launching the 'user referral' program, in all three 'sale index', i.e., 'daily purchase activity', 'daily money spent', 'daily customers', are all increased, however,  <span style='color:orange;font-size:1.5em;font-weight:bold'>none of those increment are significant</span>. (by using a ** 0.05 ** significant level)

Therefore, the program [doesn't seem have significant impacts to the whole company as a whole](#whole_result).

In [25]:
daily_stat_bycountry = data.groupby('country').apply(daily_statistics)
daily_stat_bycountry

avg_before avg_after avg_diff     p_value
country                                                      
CA      n_purchase     177.143       160 -17.1429    0.233985
        n_customer     173.286   159.179 -14.1071    0.268256
        total_spent    7468.43   7880.43      412    0.351704
CH      n_purchase     26.8214   17.0714    -9.75  0.00307243
        n_customer     26.7143   17.0714 -9.64286  0.00314248
        total_spent    1536.32   1023.89 -512.429  0.00694125
DE      n_purchase     232.143   164.036 -68.1071    0.011798
        n_customer     224.964    163.25 -61.7143   0.0156648
        total_spent    9856.75   8013.96 -1842.79   0.0814595
ES      n_purchase     156.607     194.5  37.8929   0.0726382
        n_customer     153.393   193.214  39.8214   0.0579535
        total_spent    6648.64   8660.57  2011.93   0.0375216
FR      n_purchase     244.143   305.714  61.5714   0.0589957
        n_customer       236.5   302.536  66.0357   0.0411235
        total_spent    10385.2     13635  3249.75   0.0318428
IT      n_purchase     180.857   227.929  47.0714   0.0574544
        n_customer     176.536   226.107  49.5714   0.0439105
        total_spent    7651.57   10193.4  2541.86     0.02573
MX      n_purchase     126.464       164  37.5357   0.0324303
        n_customer     124.393   163.107  38.7143   0.0262032
        total_spent    4975.46   7033.21  2057.75  0.00967015
UK      n_purchase     264.286   289.036    24.75    0.261183
        n_customer     255.571   286.321    30.75    0.204398
        total_spent    11213.5   14196.4  2982.89   0.0484899
US      n_purchase     282.286   263.429 -18.8571    0.307801
        n_customer     273.179   261.107 -12.0714     0.36886
        total_spent      11921   13077.5  1156.43    0.248874

#### daily spent change in each country

from above result, we know <span style='color:blue;font-weight:bold'>'User Referral' program has different effect in different countries</span>. The program boosts the sales in some country, but in some other countries, <span style='color:red;font-weight:bold'>it even decrease the sales.</span>

In [26]:
daily_stat_bycountry.xs('total_spent',level = 1).sort_values(by = 'p_value')

,avg_before,avg_after,avg_diff,p_value
country,,,,
CH,1536.32,1023.89,-512.429,0.00694125
MX,4975.46,7033.21,2057.75,0.00967015
IT,7651.57,10193.4,2541.86,0.02573
FR,10385.2,13635,3249.75,0.0318428
ES,6648.64,8660.57,2011.93,0.0375216
UK,11213.5,14196.4,2982.89,0.0484899
DE,9856.75,8013.96,-1842.79,0.0814595
US,11921,13077.5,1156.43,0.248874
CA,7468.43,7880.43,412,0.351704


from above result, if we loose the significant level=0.1, then
* <span style='color:orange;font-weight:bold'>daily spent in 'CH' and 'DE' are significantly decreased.</span>
* <span style='color:orange;font-weight:bold'>'MX','IT','FR','ES','UK', their daily spent are significant increased.</span>
* <span style='color:orange;font-weight:bold'>'US' and 'CA' has some improvement in daily spent, but NOT significant.</span>

#### daily customer change in each country

In [27]:
daily_stat_bycountry.xs('n_customer',level = 1).sort_values(by = 'p_value')

,avg_before,avg_after,avg_diff,p_value
country,,,,
CH,26.7143,17.0714,-9.64286,0.00314248
DE,224.964,163.25,-61.7143,0.0156648
MX,124.393,163.107,38.7143,0.0262032
FR,236.5,302.536,66.0357,0.0411235
IT,176.536,226.107,49.5714,0.0439105
ES,153.393,193.214,39.8214,0.0579535
UK,255.571,286.321,30.75,0.204398
CA,173.286,159.179,-14.1071,0.268256
US,273.179,261.107,-12.0714,0.36886


from above result, 
* <span style='color:orange;font-weight:bold'>daily customers in 'CH' and 'DE' are significantly decreased.</span>
* <span style='color:orange;font-weight:bold'>'MX','IT','FR','ES', their daily customers are significant increased.

#### daily transaction change in each country

In [31]:
daily_stat_bycountry.xs('n_purchase',level = 1).sort_values(by = 'p_value')

,avg_before,avg_after,avg_diff,p_value
country,,,,
CH,26.8214,17.0714,-9.75,0.00307243
DE,232.143,164.036,-68.1071,0.011798
MX,126.464,164,37.5357,0.0324303
IT,180.857,227.929,47.0714,0.0574544
FR,244.143,305.714,61.5714,0.0589957
ES,156.607,194.5,37.8929,0.0726382
CA,177.143,160,-17.1429,0.233985
UK,264.286,289.036,24.75,0.261183
US,282.286,263.429,-18.8571,0.307801


from above result, 
* <span style='color:orange;font-weight:bold'>daily transactions in 'CH' and 'DE' are significantly decreased.</span>
* <span style='color:orange;font-weight:bold'>'MX','IT','FR','ES', their daily transactions are significant increased.

* <span style='color:orange;font-weight:bold;font-size:1.5em'>the program fails in CH and DE, it significantly decrease the sales in these two countries.</span>
* <span style='color:orange;font-weight:bold;font-size:1.5em'>the program succeeds in 'MX','IT','FR','ES', it significantly increase the sales.</span>
* <span style='color:orange;font-weight:bold;font-size:1.5em'>the program doesn't seem have any significant effect on UK,CA,US, especially on CA and US.</span>

**Answer question 1**

Can you estimate the impact the program had on the site?

according to the analysis above, the program [doesn't seem have significant impacts to the whole company as a whole](#whole_result).

however, based on each country, I find the program has [different impact on different country](#Country-based-conclusion):
* ** the program fails in CH and DE, it significantly decrease the sales in these two countries.**
* ** the program succeeds in 'MX','IT','FR','ES', it significantly increase the sales. **
* ** the program doesn't seem have any significant effect on UK,CA,US, especially on CA and US.**

## QUESTION 2

### Based on the data, what would you suggest to do as a next step?

1. first I suggest perform more accurate A/B test ([see question 3's answer](#Answer-question-3)) and collect more data, to study the impact of the program
2. since the program has different impact in different country, I suggest studying the reason of such difference. ** for example, does the program has any cultural conflicts in CH and DE? **

## QUESTION 3

The referral program wasn't really tested in a rigorous way. It simply started on a given day for all users and you are drawing conclusions by looking at the data before and after the test started. What kinds of risks this approach presents? Can you think of a better way to test the referral program and measure its impact?

To get more accurate impact of the program, we need to perform a more careful A/B test. for example:
* during the same peroid of time
* randomly split the customers into two groups, and let only one group know the User Referral program.
* run the experiment some time, then perform the t-test to see whether some 'sale performance index' (e.g., daily spent, daily customers, daily transactions) have significant changes or not.